<a href="https://colab.research.google.com/github/BhavanaB25/SIH/blob/main/DTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import zipfile
import os

# Define paths
zip_path = "/content/drive/MyDrive/MyProject/DTI/dti.zip"
extract_path = "/content/drive/MyDrive/MyProject/DTI/DTI_Categorized"

# Check if extraction is needed
if not os.path.exists(extract_path) or not os.listdir(extract_path):
    os.makedirs(extract_path, exist_ok=True)

    # Extract ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    print(f"✅ Extraction Completed: {extract_path}")
else:
    print(f"📂 Extraction Skipped: Already Exists in {extract_path}")

# Check extracted folder contents
print(f"📂 Extracted Folders: {os.listdir(extract_path)}")


✅ Extraction Completed: /content/drive/MyDrive/MyProject/DTI/DTI_Categorized
📂 Extracted Folders: ['ADNI']


In [4]:
!pip install pydicom nibabel numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.3 MB/s eta 0:00:00


Preprocessing Code

In [8]:
import zipfile
import os
import pandas as pd
import pydicom
import numpy as np
import cv2
from pathlib import Path
from tqdm import tqdm


# **Step 2: Locate 'ADNI' Folder Inside Extraction Path**
adni_folder = os.path.join(extract_path, "ADNI")
if not os.path.exists(adni_folder):
    print("❌ Error: 'ADNI' folder not found inside `DTI_Categorized`")
    exit()

# **Step 3: Load Patient Categories from CSV**
csv_path = "/content/drive/MyDrive/MyProject/DTI/DTI_CSV.csv"
df = pd.read_csv(csv_path)

# Convert CSV to dictionary {patient_id: category}
patient_categories = dict(zip(df["Subject_ID"], df["Diagnosis"]))

# **Step 4: Convert DICOM to PNG**
png_output_dir = "/content/drive/MyDrive/MyProject/DTI/DTI_PNG"
os.makedirs(png_output_dir, exist_ok=True)

# Loop through each patient inside 'ADNI' folder
for patient_folder in tqdm(os.listdir(adni_folder)):
    patient_id = patient_folder  # Folder name is the patient ID
    patient_path = os.path.join(adni_folder, patient_folder)

    # Skip if it's not a directory
    if not os.path.isdir(patient_path):
        continue

    # Get category (AD, CN, MCI) from CSV
    category = patient_categories.get(patient_id)
    if category is None:
        print(f"❌ Warning: Patient {patient_id} not found in CSV. Skipping...")
        continue

    # Create output directory
    category_path = os.path.join(png_output_dir, category, patient_id)
    os.makedirs(category_path, exist_ok=True)

    # **Find and process DICOM files recursively**
    for dcm_file in Path(patient_path).rglob("*.dcm"):
        try:
            dicom_data = pydicom.dcmread(dcm_file)
            img_array = dicom_data.pixel_array

            # Normalize (0-255)
            img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min()) * 255
            img_array = img_array.astype(np.uint8)

            # Convert to 3-channel image
            img_rgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)

            # Save as PNG
            output_path = os.path.join(category_path, dcm_file.stem + ".png")
            cv2.imwrite(output_path, img_rgb)

        except Exception as e:
            print(f"❌ Error processing {dcm_file}: {e}")

print("✅ Preprocessing Completed: DICOM ➝ PNG Conversion Done!")


  0%|          | 1/350 [00:06<39:13,  6.74s/it]<ipython-input-8-1bf98981dc6c>:54: RuntimeWarning: overflow encountered in scalar subtract
  img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min()) * 255
  4%|▍         | 14/350 [01:07<18:38,  3.33s/it]<ipython-input-8-1bf98981dc6c>:54: RuntimeWarning: invalid value encountered in divide
  img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min()) * 255
<ipython-input-8-1bf98981dc6c>:55: RuntimeWarning: invalid value encountered in cast
  img_array = img_array.astype(np.uint8)
 61%|██████    | 212/350 [21:54<10:58,  4.77s/it]

❌ Warning: Patient 027_S_5079 not found in CSV. Skipping...


 63%|██████▎   | 219/350 [22:38<15:46,  7.23s/it]

❌ Warning: Patient 027_S_5110 not found in CSV. Skipping...
❌ Warning: Patient 027_S_5109 not found in CSV. Skipping...


 63%|██████▎   | 222/350 [22:43<08:36,  4.04s/it]

...

❌ Warning: Patient 021_S_5237 not found in CSV. Skipping...
❌ Warning: Patient 021_S_5236 not found in CSV. Skipping...


 78%|███████▊  | 273/350 [25:13<02:44,  2.13s/it]

❌ Warning: Patient 057_S_5295 not found in CSV. Skipping...


100%|██████████| 350/350 [25:14<00:00,  4.33s/it]

❌ Warning: Patient 005_S_0610 not found in CSV. Skipping...
❌ Warning: Patient 135_S_4281 not found in CSV. Skipping...
❌ Warning: Patient 135_S_4309 not found in CSV. Skipping...
❌ Warning: Patient 135_S_4356 not found in CSV. Skipping...
❌ Warning: Patient 027_S_6001 not found in CSV. Skipping...
❌ Warning: Patient 027_S_6002 not found in CSV. Skipping...
❌ Warning: Patient 027_S_0120 not found in CSV. Skipping...
❌ Warning: Patient 021_S_0626 not found in CSV. Skipping...
❌ Warning: Patient 021_S_0178 not found in CSV. Skipping...
❌ Warning: Patient 099_S_6025 not found in CSV. Skipping...
❌ Warning: Patient 099_S_6016 not found in CSV. Skipping...
❌ Warning: Patient 021_S_0337 not found in CSV. Skipping...
❌ Warning: Patient 027_S_6034 not found in CSV. Skipping...
❌ Warning: Patient 127_S_1427 not found in CSV. Skipping...
❌ Warning: Patient 099_S_6038 not found in CSV. Skipping...
❌ Warning: Patient 127_S_0259 not found in CSV. Skipping...
❌ Warning: Patient 129_S_6082 not found 

In [13]:
import os
import random
import torch
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Paths
input_dir = "/content/drive/MyDrive/MyProject/DTI/DTI_PNG"  # Original PNG folder
output_dir = "/content/drive/MyDrive/MyProject/DTI/DTI_Augmented_PNG"  # Folder for augmented images
os.makedirs(output_dir, exist_ok=True)

# Define Augmentations
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.GaussianBlur(kernel_size=3),
])

# Process Each Category (AD, CN, MCI)
for category in ["AD", "CN", "MCI"]:
    category_input_dir = os.path.join(input_dir, category)
    category_output_dir = os.path.join(output_dir, category)
    os.makedirs(category_output_dir, exist_ok=True)

    if not os.path.exists(category_input_dir):
        print(f"❌ Warning: {category} folder not found. Skipping...")
        continue

    print(f"🔹 Processing Category: {category}")

    # Process Each Patient
    for patient_id in tqdm(os.listdir(category_input_dir)):
        patient_input_dir = os.path.join(category_input_dir, patient_id)
        patient_output_dir = os.path.join(category_output_dir, patient_id)
        os.makedirs(patient_output_dir, exist_ok=True)

        if not os.path.isdir(patient_input_dir):
            continue

        # Process Each Image
        for img_name in os.listdir(patient_input_dir):
            img_path = os.path.join(patient_input_dir, img_name)

            # Load Image
            image = Image.open(img_path).convert("RGB")

            # Save Original Image
            image.save(os.path.join(patient_output_dir, img_name))

            # Apply Augmentation (Save 3 Augmented Versions)
            for i in range(3):
                augmented_image = augmentation_transforms(image)
                augmented_image.save(os.path.join(patient_output_dir, f"aug_{i}_{img_name}"))

    print(f"✅ Data Augmentation Completed for {category}!")


🔹 Processing Category: AD


100%|██████████| 49/49 [15:17<00:00, 18.73s/it]


✅ Data Augmentation Completed for AD!
🔹 Processing Category: CN


100%|██████████| 69/69 [35:38<00:00, 31.00s/it]


✅ Data Augmentation Completed for CN!
🔹 Processing Category: MCI


100%|██████████| 131/131 [1:08:02<00:00, 31.17s/it]

✅ Data Augmentation Completed for MCI!


In [11]:
!pip install torch torchvision tqdm pillow


In [ ]:
import os
import nibabel as nib
import numpy as np
import pydicom
import shutil

# Define paths
base_dir = "/content/drive/MyDrive/MyProject/DTI/DTI_Categorized"
preprocessed_dir = "/content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed"
os.makedirs(preprocessed_dir, exist_ok=True)

# Categories
categories = ["AD", "CN", "MCI"]

# Ensure preprocessed category folders exist
for category in categories:
    os.makedirs(os.path.join(preprocessed_dir, category), exist_ok=True)

# Function to convert DICOM to NIfTI
def dicom_to_nifti(dicom_folder, output_filename):
    dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith(".dcm")]

    if not dicom_files:
        return None  # No DICOM files found

    dicom_files.sort()  # Ensure files are ordered correctly

    # Read DICOM images
    slices = [pydicom.dcmread(f) for f in dicom_files]
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))  # Sort by slice position

    # Convert to NumPy array
    image_array = np.stack([s.pixel_array for s in slices], axis=-1)

    # Normalize intensity values
    image_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))

    # Save as NIfTI
    nifti_img = nib.Nifti1Image(image_array, np.eye(4))  # Identity affine matrix
    nib.save(nifti_img, output_filename)
    return output_filename

# Process each category
processed_count = 0
failed_patients = []

for category in categories:
    category_path = os.path.join(base_dir, category)
    output_category_path = os.path.join(preprocessed_dir, category)

    for patient_id in os.listdir(category_path):
        patient_path = os.path.join(category_path, patient_id)

        if not os.path.isdir(patient_path):
            continue

        # Convert DICOM to NIfTI
        output_filename = os.path.join(output_category_path, f"{patient_id}.nii.gz")
        result = dicom_to_nifti(patient_path, output_filename)

        if result:
            print(f"✅ Processed {patient_id} ({category}) → {output_filename}")
            processed_count += 1
        else:
            print(f"❌ Failed: {patient_id}")
            failed_patients.append(patient_id)

# Summary
print(f"\n✅ Preprocessing Complete: {processed_count} patients processed")
print(f"❌ Failed Cases: {len(failed_patients)} (First 10): {failed_patients[:10]}")


✅ Processed 016_S_4009 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/016_S_4009.nii.gz
✅ Processed 094_S_4089 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/094_S_4089.nii.gz
✅ Processed 003_S_4136 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/003_S_4136.nii.gz
✅ Processed 003_S_4152 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/003_S_4152.nii.gz
✅ Processed 003_S_4142 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/003_S_4142.nii.gz


<ipython-input-5-0cbf7fbd609b>:36: RuntimeWarning: overflow encountered in scalar subtract
  image_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))


✅ Processed 098_S_4215 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/098_S_4215.nii.gz
✅ Processed 098_S_4201 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/098_S_4201.nii.gz
✅ Processed 094_S_4282 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/094_S_4282.nii.gz
✅ Processed 029_S_4307 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/029_S_4307.nii.gz
✅ Processed 016_S_4353 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/016_S_4353.nii.gz
✅ Processed 109_S_4378 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/109_S_4378.nii.gz
✅ Processed 003_S_4373 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/003_S_4373.nii.gz
✅ Processed 126_S_4494 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/126_S_4494.nii.gz
✅ Processed 127_S_4500 (AD) → /content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed/AD/127_S_4500.nii.gz
✅ Processed 007_S_4568 (AD) → /content/drive/MyDrive/My

In [9]:
import shutil
import os
from tqdm import tqdm

# Define Paths
source_png_dir = "/content/DTI_PNG"  # Path where PNG images are stored
destination_drive_dir = "/content/drive/MyDrive/MyProject/DTI/DTI_PNG"  # Destination in Google Drive

# Ensure the destination folder exists
os.makedirs(destination_drive_dir, exist_ok=True)

# Move all PNG images to Google Drive
for root, _, files in os.walk(source_png_dir):
    for file in tqdm(files, desc="Saving PNGs to Drive"):
        if file.endswith(".png"):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(destination_drive_dir, file)
            shutil.move(src_file, dst_file)

print(f"✅ All PNG images saved to: {destination_drive_dir}")


✅ All PNG images saved to: /content/drive/MyDrive/MyProject/DTI/DTI_PNG


CNN Model(3D ResNet-50, 3D DenseNet-121, 3D EfficientNet-B0)

In [10]:
!pip install torch torchvision tqdm pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install monai numpy nibabel torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import torch
import torch.nn as nn
from monai.networks.nets import ResNet, DenseNet, EfficientNetBN

# Define Feature Extraction Model
class FeatureExtractor(nn.Module):
    def __init__(self, num_classes=3):
        super(FeatureExtractor, self).__init__()

        # 3D ResNet (ResNet50) - Fixed block type
        self.resnet3d = ResNet(
            block="bottleneck",  # ✅ Fixed (was "BOTTLE")
            layers=[3, 4, 6, 3],
            block_inplanes=[64, 128, 256, 512],
            spatial_dims=3, n_input_channels=1, num_classes=num_classes
        )

        # 3D DenseNet (DenseNet121)
        self.densenet3d = DenseNet(
            spatial_dims=3, in_channels=1, out_channels=num_classes
        )

        # 3D EfficientNet (EfficientNet-B0)
        self.efficientnet3d = EfficientNetBN(
            model_name="efficientnet-b0", spatial_dims=3, in_channels=1, num_classes=num_classes
        )

        # Fully connected layer to combine extracted features
        self.fc = nn.Linear(num_classes * 3, num_classes)  # Combining 3 model outputs

    def forward(self, x):
        resnet_features = self.resnet3d(x)
        densenet_features = self.densenet3d(x)
        efficientnet_features = self.efficientnet3d(x)

        # Concatenate features
        combined_features = torch.cat((resnet_features, densenet_features, efficientnet_features), dim=1)

        # Fully connected layer
        output = self.fc(combined_features)
        return output

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeatureExtractor(num_classes=3).to(device)

# Print model summary
print(model)


FeatureExtractor(
  (resnet3d): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)


In [ ]:
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FeatureExtractor(num_classes=3).to(device)

# Print summary for each individual model
print("===== ResNet3D Summary =====")
summary(model.resnet3d, (1, 64, 64, 64))  # Adjust input size as needed

print("===== DenseNet3D Summary =====")
summary(model.densenet3d, (1, 64, 64, 64))  # Adjust input size

print("===== EfficientNet3D Summary =====")
summary(model.efficientnet3d, (1, 64, 64, 64))  # Adjust input size


===== ResNet3D Summary =====
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 64, 64, 64]          21,952
       BatchNorm3d-2       [-1, 64, 64, 64, 64]             128
              ReLU-3       [-1, 64, 64, 64, 64]               0
         MaxPool3d-4       [-1, 64, 32, 32, 32]               0
            Conv3d-5       [-1, 64, 32, 32, 32]           4,096
       BatchNorm3d-6       [-1, 64, 32, 32, 32]             128
              ReLU-7       [-1, 64, 32, 32, 32]               0
            Conv3d-8       [-1, 64, 32, 32, 32]         110,592
       BatchNorm3d-9       [-1, 64, 32, 32, 32]             128
             ReLU-10       [-1, 64, 32, 32, 32]               0
           Conv3d-11      [-1, 256, 32, 32, 32]          16,384
      BatchNorm3d-12      [-1, 256, 32, 32, 32]             512
           Conv3d-13      [-1, 256, 32, 32, 32]          16,640
      Batc

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np

class DTIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = [f for f in os.listdir(root_dir) if f.endswith('.npy')]  # Assuming NIfTI converted to .npy

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        file_path = os.path.join(self.root_dir, file_name)

        image = np.load(file_path)  # Assuming .npy format
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dim

        # Extract label from filename (Assume filenames contain label info, e.g., 'AD_001.npy')
        label_str = file_name.split("_")[0]
        label_map = {"CN": 0, "MCI": 1, "AD": 2}  # Modify if needed
        label = torch.tensor(label_map[label_str], dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# Define dataset path
dataset_path = "/content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed"

# Load dataset
test_dataset = DTIDataset(root_dir=dataset_path)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

print(f"✅ Dataset loaded! Total samples: {len(test_dataset)}")


✅ Dataset loaded! Total samples: 0


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("===== ResNet3D Accuracy =====")
resnet_acc = compute_accuracy(model.resnet3d, test_loader, device)
print(f"ResNet3D Accuracy: {resnet_acc:.2f}%")


===== ResNet3D Accuracy =====
ResNet3D Accuracy: 0.00%


In [ ]:
print(globals().keys())  # Lists all variables in the notebook


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'drive', '_i2', 'zipfile', 'os', 'zip_path', 'extract_path', 'zip_ref', '_i3', 'shutil', 'base_dir', 'source_dir', 'categories', 'category', 'category_path', 'find_dicom_folder', 'pd', 'csv_path', 'df', 'patient_category_map', 'moved_patients', 'not_found_patients', 'patient_id', 'patient_path', 'dicom_folder', 'diagnosis', 'dest_path', 'file', 'src_file', 'dest_file', '_i4', 'nib', 'np', '_i5', '_exit_code', '_i6', 'pydicom', 'preprocessed_dir', 'dicom_to_nifti', 'processed_count', 'failed_patients', 'output_category_path', 'output_filename', 'result', '_i7', '_i8', 'torch', 'nn', 'models', 'resnet', 'densenet', 'efficientnet', 'device', 'data_dir', 'output_features_dir', 'FeatureExtractor', '_i9', 'model', 'feature_save_path', 'patient_features', 'patient_ids', 'patient'

In [ ]:
# If you loaded a model from a checkpoint, reload it
import torch
import torchvision.models as models

resnet3d = models.video.r3d_18(pretrained=True)  # Example for a 3D ResNet-18
resnet3d = resnet3d.to("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:01<00:00, 83.9MB/s]


In [ ]:
for name, param in resnet3d.named_parameters():
    print(f"{name} - Requires Grad: {param.requires_grad}")


stem.0.weight - Requires Grad: True
stem.1.weight - Requires Grad: True
stem.1.bias - Requires Grad: True
layer1.0.conv1.0.weight - Requires Grad: True
layer1.0.conv1.1.weight - Requires Grad: True
layer1.0.conv1.1.bias - Requires Grad: True
layer1.0.conv2.0.weight - Requires Grad: True
layer1.0.conv2.1.weight - Requires Grad: True
layer1.0.conv2.1.bias - Requires Grad: True
layer1.1.conv1.0.weight - Requires Grad: True
layer1.1.conv1.1.weight - Requires Grad: True
layer1.1.conv1.1.bias - Requires Grad: True
layer1.1.conv2.0.weight - Requires Grad: True
layer1.1.conv2.1.weight - Requires Grad: True
layer1.1.conv2.1.bias - Requires Grad: True
layer2.0.conv1.0.weight - Requires Grad: True
layer2.0.conv1.1.weight - Requires Grad: True
layer2.0.conv1.1.bias - Requires Grad: True
layer2.0.conv2.0.weight - Requires Grad: True
layer2.0.conv2.1.weight - Requires Grad: True
layer2.0.conv2.1.bias - Requires Grad: True
layer2.0.downsample.0.weight - Requires Grad: True
layer2.0.downsample.1.weigh

In [ ]:
import torch.optim as optim
import torch.nn as nn

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(resnet3d.parameters(), lr=0.001, weight_decay=1e-5)


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import os
import numpy as np
import nibabel as nib  # If your images are in .nii format

class DTI_Dataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data_info = pd.read_csv(csv_file)  # Load labels CSV

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data_info.iloc[idx, 0])  # Image file path
        image = nib.load(img_path).get_fdata()  # Load DTI image
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dim
        label = torch.tensor(self.data_info.iloc[idx, 1], dtype=torch.long)  # Get label

        if self.transform:
            image = self.transform(image)  # Apply transformations

        return image, label


In [ ]:
from sklearn.model_selection import train_test_split

# Paths
dti_root = "/content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed"
csv_file = "/content/drive/MyDrive/MyProject/DTI/DTI_CSV.csv"

# Load dataset
dataset = DTI_Dataset(dti_root, csv_file)

# Split into train/test (80-20 split)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import os
import pandas as pd
import nibabel as nib

class DTI_Dataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data_info = pd.read_csv(csv_file)  # Load labels CSV

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        patient_id = str(self.data_info.iloc[idx, 0])  # Patient ID from CSV
        label = self.data_info.iloc[idx, 1]  # Diagnosis (AD, CN, MCI)

        # Search for the correct file inside AD, CN, MCI folders
        subfolders = ["AD", "CN", "MCI"]
        img_path = None
        for subfolder in subfolders:
            full_path = os.path.join(self.root_dir, subfolder, f"{patient_id}.nii.gz")
            if os.path.exists(full_path):
                img_path = full_path
                break

        if img_path is None:
            raise FileNotFoundError(f"File for patient {patient_id} not found!")

        # Load the NIfTI file
        image = nib.load(img_path).get_fdata()
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dim

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)


In [ ]:
class DTI_Dataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data_info = pd.read_csv(csv_file)  # Load labels CSV

        # Convert labels to numbers (AD=2, MCI=1, CN=0)
        self.label_mapping = {'CN': 0, 'MCI': 1, 'AD': 2}
        self.data_info['label'] = self.data_info['Diagnosis'].map(self.label_mapping)

    def __getitem__(self, idx):
        patient_id = str(self.data_info.iloc[idx, 0])  # Patient ID
        label = self.data_info.iloc[idx, 2]  # Numeric label

        # Find the correct file inside AD, CN, MCI folders
        subfolders = ["AD", "CN", "MCI"]
        img_path = None
        for subfolder in subfolders:
            full_path = os.path.join(self.root_dir, subfolder, f"{patient_id}.nii.gz")
            if os.path.exists(full_path):
                img_path = full_path
                break

        if img_path is None:
            raise FileNotFoundError(f"File for patient {patient_id} not found!")

        # Load the NIfTI file
        image = nib.load(img_path).get_fdata()
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dim

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)  # Now label is an integer


In [ ]:
import pandas as pd

csv_path = "/content/drive/MyDrive/MyProject/DTI/DTI_CSV.csv"  # Update path if needed
df = pd.read_csv(csv_path)

# Display the first few rows
print(df.head())

# Print column names
print(df.columns)


   Subject_ID Diagnosis  Image_UID
0  003_S_0908       MCI     404525
1  003_S_1074       MCI     487988
2  003_S_2374       MCI     234455
3  003_S_2374       MCI     502312
4  003_S_4081        CN     244118
Index(['Subject_ID', 'Diagnosis', 'Image_UID'], dtype='object')


In [ ]:
csv_path = "/content/drive/MyDrive/MyProject/DTI/DTI_CSV.csv"
with open(csv_path, "r") as f:
    print(f.readlines()[:10])  # Show first 10 lines


['\n']


In [ ]:
df.to_csv("/content/drive/MyDrive/MyProject/DTI_CSV.csv", index=False)
print("✅ CSV file recovered and saved to Google Drive!")


✅ CSV file recovered and saved to Google Drive!


In [ ]:
dataset = DTI_Dataset(csv_file="/content/drive/MyDrive/MyProject/DTI/DTI_CSV.csv",
                       root_dir="/content/drive/MyDrive/MyProject/DTI/DTI_Preprocessed")

sample_img, sample_label = dataset[0]

print(f"Sample image shape: {sample_img.shape}")  # Should be (1, 256, 256, 59)
print(f"Sample label: {sample_label}")  # Should be 0, 1, or 2


EmptyDataError: No columns to parse from file